Exercise: Try a Support Vector Machine regressor (sklearn.svm.SVR) with various hyperparameters, such as kernel="linear" (with various values for the C hyperparameter) or kernel="rbf" (with various values for the C and gamma hyperparameters). Note that SVMs don't scale well to large datasets, so you should probably train your model on just the first 5,000 instances of the training set and use only 3-fold cross-validation, or else it will take hours. Don't worry about what the hyperparameters mean for now (see the SVM notebook if you're interested). How does the best SVR predictor perform?

### LIBRARIES

In [12]:
import os
import sys
sys.path.append(os.path.abspath('.'))  # Dodanie bieżącego katalogu
sys.path.append(os.path.abspath('..'))
from Functions import load_housing_data, shuffle_and_split, split_data_with_id_hash
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
housing = load_housing_data()
from sklearn.compose import make_column_selector, make_column_transformer, ColumnTransformer
from Classes import StandardScalerClone, ClusterSimilarity
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer



ModuleNotFoundError: No module named 'Classes'

In [13]:
import sys
import os

print("Ścieżka Pythona:")
print(sys.path)

print("\nPliki w bieżącym katalogu:")
print(os.listdir('.'))

print("\nPliki w katalogu wyżej:")
print(os.listdir('..'))


Ścieżka Pythona:
['C:\\Users\\macie\\AppData\\Local\\Programs\\Python\\Python313\\python313.zip', 'C:\\Users\\macie\\AppData\\Local\\Programs\\Python\\Python313\\DLLs', 'C:\\Users\\macie\\AppData\\Local\\Programs\\Python\\Python313\\Lib', 'C:\\Users\\macie\\AppData\\Local\\Programs\\Python\\Python313', 'c:\\Users\\macie\\OneDrive\\Desktop\\Nauka2\\.venv', '', 'c:\\Users\\macie\\OneDrive\\Desktop\\Nauka2\\.venv\\Lib\\site-packages', 'c:\\Users\\macie\\OneDrive\\Desktop\\Nauka2\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\macie\\OneDrive\\Desktop\\Nauka2\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\macie\\OneDrive\\Desktop\\Nauka2\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\macie\\OneDrive', 'c:\\Users\\macie']

Pliki w bieżącym katalogu:
['.849C9593-D756-4E56-8D6E-42412F2A707B', 'datasets', 'Desktop', 'desktop.ini', 'Documents', 'Magazyn osobisty.lnk', 'Pictures', 'Załączniki e-mail']

Pliki w katalogu wyżej:
['.gitconfig', '.gk', '.ipython', '.matplotlib', '.vscode',

### DATA DOWNLOAD AND SPLIT
Use transformation from chapter_2_notebook, in this exercies i only want to check how SVM perfrom, without going into details.

In [6]:
housing = load_housing_data()
housing['income_cat'] = pd.cut(housing['median_income'], 
                               bins = [0., 1.5, 3.0, 4.5, 6., np.inf ], 
                               labels = [1, 2, 3, 4, 5])

strat_train_set, strat_test_set = train_test_split(housing, test_size = 0.2, stratify = housing['income_cat'], random_state = 42)

for set_ in (strat_test_set, strat_train_set):
    set_.drop('income_cat', axis = 1, inplace = True)

housing = strat_test_set.copy()


In [ ]:
def column_ratio(X):
    return X[ :, [0]] / X[ :, [1]]

def ratio_name(function_transformer, feature_names_in):
    return ['ratio']

def ratio_pipeline():
    return make_pipeline(
        SimpleImputer(strategy = 'median'),
        FunctionTransformer(column_ratio, feature_names_out = ratio_name),
        StandardScaler()
    )

log_pipeline = make_pipeline(
    SimpleImputer(strategy = 'median'),
    FunctionTransformer(np.log, feature_names_out = 'one-to-one'),
    StandardScaler()
)

cluster_simil = ClusterSimilarity(n_clusters = 10, gamma = 1, random_state = 42)
default_num_pipeline = make_pipeline(SimpleImputer(strategy = 'median'))
preprocessing = ColumnTransformer([
    ('bedroom_ratio', ratio_pipeline(), ['total_bedrooms', 'total_rooms']), 
    ('rooms_per_family', ratio_pipeline(), ['total_rooms', 'households']), 
    ('people_per_house', ratio_pipeline(), ['population', 'households']),
    ('log', log_pipeline, ['total_bedrooms', 'total_rooms', 'population', 'households', 'median_income']), 
    ('geo', cluster_simil, ['latitude', 'longitude']), 
    ('cat', cat_pipeline, make_column_selector(dtype_include=object))
],
remainder=default_num_pipeline)


housing_prepared = preprocessing.fit_transform(housing)